# ECSE 415 Assignment 2, Question 3
### Alex Hale | 260672475 | October 20, 2019

## Object Detection
### Step 1 - Training

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [5]:
def hog(img):
    cell_size = (4, 4)  # h x w in pixels
    block_size = (2, 2)  # h x w in cells
    nbins = 9  # number of orientation bins
    hog = cv2.HOGDescriptor(_winSize=(img.shape[1] // cell_size[1] * cell_size[1],
                                      img.shape[0] // cell_size[0] * cell_size[0]),
                            _blockSize=(block_size[1] * cell_size[1],
                                        block_size[0] * cell_size[0]),
                            _blockStride=(cell_size[1], cell_size[0]),
                            _cellSize=(cell_size[1], cell_size[0]),
                            _nbins=nbins)

    n_cells = (img.shape[0] // cell_size[0], img.shape[1] // cell_size[1])

    # Compute HoG features
    hog_feats = hog.compute(img) \
                   .reshape(n_cells[1] - block_size[1] + 1,
                            n_cells[0] - block_size[0] + 1,
                            block_size[1], block_size[0], nbins) \
                   .transpose((1, 0, 3, 2, 4))  # index blocks by rows first
    
    return (hog_feats)


#     # Preview
#     plt.figure(figsize = (20,100))
#     plt.subplot(151)
#     plt.imshow(img, cmap='gray')
#     plt.title("Original Image"), plt.xticks([]), plt.yticks([])

#     plt.subplot(152)
#     plt.pcolor(hog_feats[:, :, 0,0,0])
#     plt.gca().invert_yaxis()
#     plt.gca().set_aspect('equal', adjustable='box')
#     plt.title("HOG bin = 0, block = 1"), plt.xticks([]), plt.yticks([])

#     plt.subplot(153)
#     plt.pcolor(hog_feats[:, :, 0,1,0])
#     plt.gca().invert_yaxis()
#     plt.gca().set_aspect('equal', adjustable='box')
#     plt.title("HOG bin = 0, block = 2"), plt.xticks([]), plt.yticks([])

#     plt.subplot(154)
#     plt.pcolor(hog_feats[:, :, 1,0,0])
#     plt.gca().invert_yaxis()
#     plt.gca().set_aspect('equal', adjustable='box')
#     plt.title("HOG bin = 0, block = 3"), plt.xticks([]), plt.yticks([])

#     plt.subplot(155)
#     plt.pcolor(hog_feats[:, :, 1,1,0])
#     plt.gca().invert_yaxis()
#     plt.gca().set_aspect('equal', adjustable='box')
#     plt.title("HOG bin = 0, block = 4"), plt.xticks([]), plt.yticks([])
#     plt.colorbar(fraction=0.04)
#     plt.show()

In [15]:
cars = []
cars_flipped = []
feat_vecs = []
feat_vecs_flipped = []
for i in range(14):
    cars.append(cv2.cvtColor(cv2.imread("car" + str(i+1) + ".jpg"), cv2.COLOR_BGR2GRAY))
    cv2.resize(cars[i], (128, 128))
    feat_vecs.append(hog(cars[i]))
    cars_flipped.append(cv2.flip(cars[i], 0))
    feat_vecs_flipped.append(hog(cars_flipped[i]))

# TODO calculate and store the mean feature vector across the training images

# TODO display 9 orientation channels of the mean feature maps for the first block

### Step 2 - Testing